https://github.com/pranavphoenix/BiLSTM-POS-Tagging/blob/main/BiLSTM_POS_Tagging.ipynb

TODO:
- Guardare creazione dizionario, bisogna rispettare i punti dell'assignment;
- Non togliere punctuation e symbols ma evitare di utilizzarli nel calcolo delle metriche, magari utilizzando l'array di pesi 'sample_weight' che si trova nell'altro notebook;
- Provare se i risultati migliorano con preprocessing (e.g. lowerando le parole);
- Aggiustare il notebook perché fa cagare;

# Assignment 1

**Due to**: 11/01/2022 (dd/mm/yyyy)

If you deliver it by 11/12/2021 your assignment will be graded by 11/01/2022.


**Credits**: Andrea Galassi, Federico Ruggeri, Paolo Torroni

**Summary**: Part-of Speech (POS) tagging as Sequence Labelling using Recurrent Neural Architectures

# Execution
## 0. Utils

In [1]:
import nltk
import numpy as np
import os
import re
import pandas as pd
import seaborn as sns
from collections import defaultdict

from keras import backend as K
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed
from keras.layers import Embedding, Activation
from keras.models import Sequential
from keras.optimizers import Adam
from keras_preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

import urllib.request
import zipfile
import progressbar
from IPython.display import display_html
from itertools import chain,cycle

# Preprocessing of words
def pre_process(df,string):
    def text_pre_process(text):
        ret = re.sub('RT @(.)+?:\s|(&#[0-9]+;)|@([\w\-]+)|(#)\S+|(http)s?\S+|&gt;|^\s+|\b\s+|\n', '', text)
        ret = re.sub('\s\s+|[^a-zA-Z\d\s:]' , ' ', ret).rstrip().lower()
        return ret
    return df[string].apply(text_pre_process)

# Downloading Glove Word Embeddings
pbar = None
def show_progress(block_num, block_size, total_size):
    global pbar
    if pbar is None:
        pbar = progressbar.ProgressBar(maxval=total_size)
        pbar.start()

    downloaded = block_num * block_size
    if downloaded < total_size:
        pbar.update(downloaded)
    else:
        pbar.finish()
        pbar = None

# Display dataframes
def display(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:left"><td style="vertical-align:top">'
        html_str+=f'<h4 style="text-align: left;">{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [2]:
# Downloading the dataset
nltk.download('treebank')

# Download the GloVe embeddings file
url = 'http://nlp.stanford.edu/data/glove.6B.zip'
urllib.request.urlretrieve(url, 'glove.6B.zip', show_progress)

# Extract the zip file
zip_ref = zipfile.ZipFile('glove.6B.zip', 'r')
zip_ref.extractall()
zip_ref.close()

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
100% (862182613 of 862182613) |##########| Elapsed Time: 0:02:38 Time:  0:02:38


## 1. Corpus
### 1.1 Pre-processing

From the original tags list we removed all the symbols and english punctuation plus:
- FW, Foreign Word, because there are no examples in the test set;
- UH, Interjection, because there are no examples in the test set;
- LS, List Item Marker, because there are no examples in the test set (and because it denotes symbols as well);

In [26]:
# Get the files' list
fileids = nltk.corpus.treebank.fileids()

# Get the Penn Treebank tagged sentences
train_corpus = nltk.corpus.treebank.tagged_sents(fileids[:100])
val_corpus = nltk.corpus.treebank.tagged_sents(fileids[100:150])
test_corpus = nltk.corpus.treebank.tagged_sents(fileids[150:])

# Flatten the lists
# ignore = [':', '#', '"', '$', '-LRB-', '-RRB-', ',', '.', "''", '``', 'SYM', '-NONE-']

train_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(train_corpus) for item in sublist if item[1] != '-NONE-']
val_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(val_corpus) for item in sublist if item[1] != '-NONE-']
test_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(test_corpus) for item in sublist if item[1] != '-NONE-']

# train_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(train_corpus) for item in sublist if item[1] not in ignore]
# val_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(val_corpus) for item in sublist if item[1] not in ignore]
# test_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(test_corpus) for item in sublist if item[1] not in ignore]

# train_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(train_corpus) for item in sublist]
# val_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(val_corpus) for item in sublist]
# test_corpus = [tuple(list(item)+[str(idx)]) for idx,sublist in enumerate(test_corpus) for item in sublist]

train_df = pd.DataFrame(train_corpus, columns = ['word', 'tag', 'sentence'])
train_df['word'] = train_df['word'].str.lower()
# train_df['word'] = pre_process(train_df,'word')

val_df = pd.DataFrame(val_corpus, columns = ['word', 'tag', 'sentence'])
val_df['word'] = val_df['word'].str.lower()
# val_df['word'] = pre_process(val_df,'word')

test_df = pd.DataFrame(test_corpus, columns = ['word', 'tag', 'sentence'])
test_df['word'] = test_df['word'].str.lower()
# test_df['word'] = pre_process(test_df,'word')

display(train_df.describe(), val_df.describe(), test_df.describe(), titles = [f'Training set {train_df.shape}', f'Validation set {val_df.shape}', f'Test set {test_df.shape}'])

,word,tag,sentence
count,50748,50748,50748
unique,7837,46,1963
top,",",NN,1854
freq,2570,6270,271
,word,tag,sentence
count,33260,33260,33260
unique,5596,45,1299
top,the,NN,339
freq,1670,4513,90
,word,tag,sentence


In [27]:
tags_train = sorted(list(set([x for x in train_df.tag])))
tags_val = sorted(list(set([x for x in val_df.tag])))
tags_test = sorted(list(set([x for x in test_df.tag])))

max_tags_list = max([len(tags_train),len(tags_val),len(tags_test)])

# Training set tags list
print(f'Train tags number: {len(tags_train)}')
print(f'Train tags list: {tags_train}')

exceeding_validation = [el for el in tags_train if el not in tags_val]
if exceeding_validation != []:
  print(f'\tClasses in training set for which there are no samples in validation set: {exceeding_validation}')

exceeding_test = [el for el in tags_train if el not in tags_test]

if exceeding_test != []:
  print(f'\tClasses in training set for which there are no samples in test set: {exceeding_test}')


# Validation set tags list
print(f'\nValidation tags number: {len(tags_val)}')
print(f'Validation tags list: {tags_val}')

exceeding_training = [el for el in tags_val if el not in tags_train]
if exceeding_training != []:
  print(f'\tClasses in validation set for which there are no samples in training set: {exceeding_training}')

exceeding_test = [el for el in tags_val if el not in tags_test]
if exceeding_test != []:
  print(f'\tClasses in validation set for which there are no samples in test set: {exceeding_test}')

# Validation set tags list
print(f'\nTest tags number: {len(tags_test)}')
print(f'Test tags list: {tags_test}')

exceeding_training = [el for el in tags_test if el not in tags_train]
if exceeding_training != []:
  print(f'\tClasses in test set for which there are no samples in training set: {exceeding_training}')

exceeding_val = [el for el in tags_test if el not in tags_val]
if exceeding_val != []:
  print(f'\tClasses in test set set for which there are no samples in validation set: {exceeding_val}')

# conf_mat_df = pd.DataFrame(columns=tags_train, index=tags_train)
# conf_mat_df = conf_mat_df.fillna(0)

Train tags number: 46
Train tags list: ['#', '$', "''", ',', '-LRB-', '-NONE-', '-RRB-', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
	Classes in training set for which there are no samples in validation set: ['SYM']
	Classes in training set for which there are no samples in test set: ['#', 'FW', 'LS', 'SYM', 'UH']

Validation tags number: 45
Validation tags list: ['#', '$', "''", ',', '-LRB-', '-NONE-', '-RRB-', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
	Classes in validation set for which there are no samples in test set: ['#', 'FW', 'LS', 'UH']

Test tags number: 41
Test tags 

In [29]:
# Retriving prepocessed data
X_train_raw = train_df.groupby('sentence').word.apply(list).reset_index()['word']
X_val_raw = val_df.groupby('sentence').word.apply(list).reset_index()['word']
X_test_raw = test_df.groupby('sentence').word.apply(list).reset_index()['word']

y_train_raw = train_df.groupby('sentence').tag.apply(list).reset_index()['tag']
y_val_raw = val_df.groupby('sentence').tag.apply(list).reset_index()['tag']
y_test_raw = test_df.groupby('sentence').tag.apply(list).reset_index()['tag']

# Creating sets of words and tags
words = set([word.lower() for sentence in [*X_train_raw, *X_val_raw] for word in sentence])
tags = set([tag for sentence_tags in [*y_train_raw, *y_val_raw] for tag in sentence_tags])

# Bulding vocabulary of words and tags 
word2index = {word: i + 2 for i, word in enumerate(list(words))}
word2index['-PAD-'] = 0  # 0 is assigned for padding
word2index['-OOV-'] = 1  # 1 is assigned for unknown words
tag2index = {tag: i for i, tag in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # 0 is assigned for padding

vocab_length = len(word2index)
print(f'Length vocabulary: {vocab_length}')

Length vocabulary: 10343


In [30]:
# Tokenising words and tags by their indexes in vocabulary
X_train, X_val, X_test, y_train, y_val, y_test = [], [], [], [], [], []

# Encode X
for sentence in X_train_raw:
    sent_int = []
    for word in sentence:
        try:
            sent_int.append(word2index[word.lower()])
        except KeyError:
            sent_int.append(word2index['-OOV-'])
    X_train.append(sent_int)

for sentence in X_val_raw:
    sent_int = []
    for word in sentence:
        try:
            sent_int.append(word2index[word.lower()])
        except KeyError:
            sent_int.append(word2index['-OOV-'])
    X_val.append(sent_int)

for sentence in X_test_raw:
    sent_int = []
    for word in sentence:
        try:
            sent_int.append(word2index[word.lower()])
        except KeyError:
            sent_int.append(word2index['-OOV-'])
    X_test.append(sent_int)

# Encode Y
for sent_tags in y_train_raw:
    y_train.append([tag2index[tag] for tag in sent_tags])

for sent_tags in y_val_raw:
    y_val.append([tag2index[tag] for tag in sent_tags])

for sent_tags in y_test_raw:
    y_test.append([tag2index[tag] for tag in sent_tags])

print('-Not encoded')
print('\t',X_train_raw[0]) 
print('\t',y_train_raw[0])
print('-Encoded')
print('\t',X_train[0])
print('\t',y_train[0])

-Not encoded
	 ['pierre', 'vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', '29', '.']
	 ['NNP', 'NNP', ',', 'CD', 'NNS', 'JJ', ',', 'MD', 'VB', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NNP', 'CD', '.']
-Encoded
	 [5680, 8733, 6504, 1962, 9494, 3958, 6504, 8467, 5149, 4672, 6961, 4731, 5030, 5220, 8902, 7998, 10064, 9358]
	 [30, 30, 0, 32, 26, 12, 0, 38, 36, 3, 8, 22, 3, 12, 8, 30, 32, 4]


In [31]:
lengths = [len(sentence) for sentence in [*X_train_raw,*X_val_raw,*X_test_raw]]
MAX_LENGTH = max(lengths) # maximum words in a sentence
AVG_LENGTH = int(sum(lengths)/len(lengths))

print(f'Length of longest sentence: {MAX_LENGTH}')
print(f'Average length: {AVG_LENGTH}')

# Add padding to sentences
# X_train = pad_sequences(X_train, maxlen=MAX_LENGTH, padding='post')
# X_val = pad_sequences(X_val, maxlen=MAX_LENGTH, padding='post')
# X_test = pad_sequences(X_test, maxlen=MAX_LENGTH, padding='post')

# y_train = pad_sequences(y_train, maxlen=MAX_LENGTH, padding='post')
# y_val = pad_sequences(y_val, maxlen=MAX_LENGTH, padding='post')
# y_test = pad_sequences(y_test, maxlen=MAX_LENGTH, padding='post')

X_train = pad_sequences(X_train, maxlen=AVG_LENGTH, padding='post')
X_val = pad_sequences(X_val, maxlen=AVG_LENGTH, padding='post')
X_test = pad_sequences(X_test, maxlen=AVG_LENGTH, padding='post')

y_train = pad_sequences(y_train, maxlen=AVG_LENGTH, padding='post')
y_val = pad_sequences(y_val, maxlen=AVG_LENGTH, padding='post')
y_test = pad_sequences(y_test, maxlen=AVG_LENGTH, padding='post')


print('-Padded')
print('\tX:',X_train[0])
print('\n\ty:',y_train[0])

# Create a sample weight vector where the weights for padded samples, punctuation and symbols 
# are set to 0 and the weights for the other samples are set to 1
ignore = [':', '#', '$', '-LRB-', '-RRB-', ',', '.', "''", '``', 'SYM', '-NONE-']
# sample_weight = np.ones(X_train.shape)
# for i in range(X_train.shape[0]):
#     for j in range(X_train.shape[1]):
#         if X_train[i][j] == 0 or y_train_raw[i][j] in ignore:
#             sample_weight[i][j] = 0

Length of longest sentence: 271
Average length: 25
-Padded
	X: [ 5680  8733  6504  1962  9494  3958  6504  8467  5149  4672  6961  4731
  5030  5220  8902  7998 10064  9358     0     0     0     0     0     0
     0]

	y: [30 30  0 32 26 12  0 38 36  3  8 22  3 12  8 30 32  4  0  0  0  0  0  0
  0]


In [32]:
y_train_one_hot = to_categorical(y_train, max_tags_list)
y_val_one_hot = to_categorical(y_val, max_tags_list)
y_test_one_hot = to_categorical(y_test, max_tags_list)

## 2. GloVe 
GloVe (Global Vectors for Word Representation) is a method for learning vector representations of words, called "word embeddings," from a large corpus of text. Word embeddings are numerical representations of words that capture the semantic relationships between words in a continuous, low-dimensional space. They are commonly used as input to natural language processing models, such as language translation and language modeling.

GloVe works by learning the co-occurrence statistics of words in a corpus, and using this information to learn word embeddings that capture the semantic relationships between words. The GloVe method produces word embeddings that are trained on a global corpus, as opposed to embeddings that are trained on a specific task or dataset.

There are different versions of the GloVe word embeddings, including 50-dimensional, 100-dimensional, and 200-dimensional embeddings. The 50-dimensional version of GloVe embeddings may be better in some applications because they have a lower dimensionality, which can make them easier to work with and more computationally efficient.

By using GloVe embeddings as the initial weights for a model, we can take advantage of these pre-trained word representations and fine-tune them for a specific task.

In [33]:
#Use the 300 dimensional GLove Word Embeddings
glove_dir = './'
embedding_dim = 300
embedding_dict = {} #initialize dictionary
f = open(os.path.join(glove_dir, f'glove.6B.{embedding_dim}d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_dict[word] = coefs
f.close()

print('Found %s word vectors.' % len(embedding_dict))

Found 400000 word vectors.


In [34]:
#Building the Embedding Layer 
embedding_matrix = np.zeros((vocab_length, embedding_dim))
# embedding_matrix = np.standard(0,1,(vocab_length, embedding_dim))
for word, i in word2index.items():
    embedding_vector = embedding_dict.get(word)
    if i < len(word2index):
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

## 3. Model
### 3.1 Baseline 
Bidirectional LSTM layers are able to process sequential data in both the forward and backward directions, which can allow the model to capture contextual information from both the past and the future. This can be particularly useful for natural language processing tasks, where the meaning of a word can depend on the context in which it is used.

In the context of POS tagging, TimeDistributed can be used to apply a tag prediction layer to each word in a sentence. For example, you might have an RNN that processes a sequence of words in a sentence, and at each time step, the RNN outputs a hidden state. You could then apply a TimeDistributed dense layer to the hidden states, which would allow you to predict the POS tag for each word in the sentence.

One advantage of using TimeDistributed for POS tagging is that it allows you to predict the POS tag for each word in the sentence simultaneously, rather than having to process the sentence one word at a time. This can be particularly useful when dealing with long sentences, as it can make the tagging process more efficient.

Overall, using TimeDistributed for POS tagging can help you build more accurate and efficient models for natural language processing tasks that involve sequential data.

In [44]:
def ignore_class_accuracy(classes=[0]):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
        
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32')
        for to_ignore in classes:
          ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
          matches = matches * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

# Define the model
baseline_model = Sequential(name='Baseline')

# Add the Embedding layer
# baseline_model.add(Embedding(input_dim=vocab_length, output_dim=embedding_dim, \
#                     weights=[embedding_matrix], input_length=MAX_LENGTH, trainable=False))
baseline_model.add(Embedding(input_dim=vocab_length, output_dim=embedding_dim, \
                    weights=[embedding_matrix], input_length=AVG_LENGTH, trainable=False))

# Add the Bidirectional LSTM layer
baseline_model.add(Bidirectional(LSTM(units=256, return_sequences=True)))

# Add the Dense/Fully-Connected layer
baseline_model.add(TimeDistributed(Dense(units=max_tags_list, activation='softmax')))

# Compile the model
baseline_model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy',\
                      ignore_class_accuracy([0,*[tag2index[tag] for tag in ignore]])])

# Summary
baseline_model.summary()

Model: "Baseline"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 25, 300)           3102900   
                                                                 
 bidirectional_6 (Bidirectio  (None, 25, 512)          1140736   
 nal)                                                            
                                                                 
 time_distributed_6 (TimeDis  (None, 25, 46)           23598     
 tributed)                                                       
                                                                 
Total params: 4,267,234
Trainable params: 1,164,334
Non-trainable params: 3,102,900
_________________________________________________________________


In [38]:
results_baseline = baseline_model.fit(X_train, y_train_one_hot, validation_data=(X_val, y_val_one_hot),\
                                      batch_size=128, epochs=10)

Epoch 1/10
16/16 [==============================] - 52s 3s/step - loss: 2.6958 - accuracy: 0.3435 - ignore_accuracy: 0.1563 - val_loss: 1.9748 - val_accuracy: 0.4935 - val_ignore_accuracy: 0.2600
Epoch 2/10
16/16 [==============================] - 10s 615ms/step - loss: 1.6179 - accuracy: 0.5901 - ignore_accuracy: 0.3341 - val_loss: 1.2555 - val_accuracy: 0.6909 - val_ignore_accuracy: 0.4131
Epoch 3/10
16/16 [==============================] - 9s 593ms/step - loss: 1.0265 - accuracy: 0.7493 - ignore_accuracy: 0.4629 - val_loss: 0.8678 - val_accuracy: 0.7747 - val_ignore_accuracy: 0.4877
Epoch 4/10
16/16 [==============================] - 9s 590ms/step - loss: 0.7306 - accuracy: 0.8142 - ignore_accuracy: 0.5250 - val_loss: 0.6774 - val_accuracy: 0.8219 - val_ignore_accuracy: 0.5366
Epoch 5/10
16/16 [==============================] - 9s 580ms/step - loss: 0.5819 - accuracy: 0.8486 - ignore_accuracy: 0.5556 - val_loss: 0.5739 - val_accuracy: 0.8488 - val_ignore_accuracy: 0.5616
Epoch 6/10


In [39]:
scores = baseline_model.evaluate(X_test, y_test_one_hot, return_dict = True)

# Obtain the predictions made by the model on the test set
predictions = baseline_model.predict(X_test)

21/21 [==============================] - 2s 67ms/step


In [41]:
# Convert the class probabilities into class labels
# [0,*[tag2index[tag] for tag in ignore]
predicted_labels = np.argmax(predictions, axis=-1)

true_pos = defaultdict(int)
false_pos = defaultdict(int)
false_neg = defaultdict(int)
precision = defaultdict(float)
recall = defaultdict(float)
f1score = defaultdict(float)

index2tag = {v: k for k, v in tag2index.items()}
for tag in tags_test:
  if tag not in ignore:
    for idx_sentence in range(len(y_test)):
      for idx_word in range(AVG_LENGTH):
      # for idx_word in range(MAX_LENGTH):
        if index2tag[y_test[idx_sentence][idx_word]] == tag:
          # If the predicted tag matches ground truth we increase true positive count for that tag
          if predicted_labels[idx_sentence][idx_word] == y_test[idx_sentence][idx_word]:
            true_pos[index2tag[y_test[idx_sentence][idx_word]]] += 1
          else:
          # If the predicted tag does not match ground truth we increase false negative count for that tag
          # and false positive count for the wrongly predicted tag
            false_neg[index2tag[y_test[idx_sentence][idx_word]]] += 1
            false_pos[index2tag[predicted_labels[idx_sentence][idx_word]]] += 1
    # We then compute precision, recall and F1 scores
    if true_pos[tag] != 0:
      precision[tag] = true_pos[tag] / (true_pos[tag] + false_pos[tag])
      recall[tag] = true_pos[tag] / (true_pos[tag] + false_neg[tag])
      f1score[tag] = 2 * ((precision[tag] * recall[tag]) / (precision[tag] + recall[tag])) 
    else:
      print(tag)
      print(true_pos[tag])
      print(false_pos[tag])
      print(false_neg[tag])
      print()

NNPS
0
0
35

PDT
0
0
3

RBS
0
0
2

WP$
0
0
3



In [42]:
# FORSE DOBBIAMO CONSIDERARE TUTTE LE CLASSI; ANCHE QUELLE PRESENTI SOLO NEL TRAINING TEST, DANDO COME f1 SCORE 0
for tag in tags_test:
  if tag not in ignore:
    print(f'Tag: {tag}')
    print(f'\tPrecision: {precision[tag]}')
    print(f'\tRecall: {recall[tag]}')
    print(f'\tF1-score: {f1score[tag]}')

Tag: CC
	Precision: 1.0
	Recall: 0.9737704918032787
	F1-score: 0.9867109634551495
Tag: CD
	Precision: 1.0
	Recall: 0.8103448275862069
	F1-score: 0.8952380952380952
Tag: DT
	Precision: 0.998046875
	Recall: 0.983638113570741
	F1-score: 0.990790111488124
Tag: EX
	Precision: 1.0
	Recall: 0.75
	F1-score: 0.8571428571428571
Tag: IN
	Precision: 0.9992025518341308
	Recall: 0.9357729648991785
	F1-score: 0.9664481295796375
Tag: JJ
	Precision: 0.9442446043165468
	Recall: 0.68717277486911
	F1-score: 0.7954545454545455
Tag: JJR
	Precision: 0.9285714285714286
	Recall: 0.5306122448979592
	F1-score: 0.6753246753246754
Tag: JJS
	Precision: 1.0
	Recall: 0.25
	F1-score: 0.4
Tag: MD
	Precision: 1.0
	Recall: 0.9928571428571429
	F1-score: 0.996415770609319
Tag: NN
	Precision: 0.9304973037747154
	Recall: 0.7815802717664821
	F1-score: 0.849562363238512
Tag: NNP
	Precision: 0.7955752212389381
	Recall: 0.8157894736842105
	F1-score: 0.8055555555555556
Tag: NNPS
	Precision: 0.0
	Recall: 0.0
	F1-score: 0.0
Tag: NN

In [46]:
provola = [el[1] for el in f1score.items()]

sum = 0

for p in provola:
  sum += p

print(f'MACRO F1-score: {sum / len(f1score.items())}')

MACRO F1-score: 0.7047731101335059
